# Multiband Images

In [1]:
##################################################
#
# Libraries
#





from metpy.plots    import colortables
from metpy.plots    import add_timestamp
from datetime       import datetime
from siphon.catalog import TDSCatalog
from datetime       import datetime


import numpy             as np
import os                as os
import xarray            as xr
import pandas            as pd
import pathlib           as pathlib

import metpy
import cartopy.crs       as ccrs
import cartopy.feature   as cfeature
import matplotlib.pyplot as plt
import matplotlib.patches as patches





#
##################################################

In [2]:
####################################################
####################################################
####################################################
#
# Mines Colors and Fonts
#

Mines_Blue = "#002554"


plt.rcParams.update({'text.color'      : Mines_Blue,
                     'axes.labelcolor' : Mines_Blue,
					 'axes.edgecolor'  : Mines_Blue,
					 'xtick.color'     : Mines_Blue,
					 'ytick.color'     : Mines_Blue})


#
####################################################
####################################################
####################################################

## Get Time Step here.

Go into the UCAR GOES Thredds directory and manually get the YYYMMDD and the YYYDOYhhmmssf

https://thredds.ucar.edu/thredds/catalog/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/catalog.html



In [3]:


YYYYMMDD         = "20230522"

YYYDOYhhmmssf    = "20231421541170"

file_time_string = "s" + YYYDOYhhmmssf + "_e" + YYYDOYhhmmssf + "_c" + YYYDOYhhmmssf


In [4]:
##################################################
#
# Channel Labels
#

MAINDIR = os.getcwd() +"/"
print(MAINDIR)

alpha_factor = 0.05



channel_lab = [' Channel Zero',                     #  0
               ' [0.47 µm Blue-Visible]',           #  1
               ' [0.64 µm Red-Visible]',            #  2
               ' [0.86 µm Vegetation Near-IR]',     #  3
               ' [1.37 µm Cirrus Near-IR]',         #  4
               ' [1.6 µm Snow/Ice Near-IR]',        #  5
               ' [2.2 µm Cloud Particle Near-IR]',  #  6
               ' [3.9 µm Middle Infrared]',         #  7
               ' [6.2 µm Upper-Level Water Vapor]', #  8
               ' [6.9 µm Mid-Level Water Vapor]',   #  9
               ' [7.3 µm Low-Level Water Vapor]',   # 10
               ' [8.4 µm Cloud-Top Infrared]',      # 11
               ' [9.6 µm Ozone Infrared]',          # 12
               ' [10.3 µm Clean IR Window]',        # 13
               ' [11.2 µm Middle IR Window]',       # 14
               ' [12.3 µm Dirty IR Window]',        # 15
               ' [13.3 µm CO₂ Infrared]']           # 16

channel_wavelengths = [0,0.47,0.64,0.86,1.37,1.6,2.2,3.9,6.2,6.9,7.3,8.4,9.6,10.3,11.2,12.3,13.3]

channel_names = ['Channel Zero',                     #  0
               'Blue-Visible',           #  1
               'Red-Visible',            #  2
               'Vegetation Near-IR',     #  3
               'Cirrus Near-IR',         #  4
               'Snow/Ice Near-IR',        #  5
               'Cloud Particle Near-IR',  #  6
               'Middle Infrared',         #  7
               'Upper-Level Water Vapor', #  8
               'Mid-Level Water Vapor',   #  9
               'Low-Level Water Vapor',   # 10
               'Cloud-Top Infrared',      # 11
               'Ozone Infrared',          # 12
               'Clean IR Window',        # 13
               'Middle IR Window',       # 14
               'Dirty IR Window',        # 15
               'CO₂ Infrared']           # 16


png_file_dir3  = "./temp_files_sat_sodak/"



imin_rap_tir =  551 # i_rap-250
imax_rap_tir = 1051 # i_rap+250
jmin_rap_tir =    0 # j_rap-250+19
jmax_rap_tir =  500 # j_rap+250-19

imin_rap_vis = 1103*2 # np.argmin(np.abs(x_vis-x_min_t).values)
imax_rap_vis = 2102*2 # np.argmin(np.abs(x_vis-x_max_t).values)
jmin_rap_vis =    0*2 # np.argmin(np.abs(y_vis-y_min_t).values)
jmax_rap_vis = 1000*2 # np.argmin(np.abs(y_vis-y_max_t).values)

vis_ny =  6000
vis_nx = 10000


imin_rap_frac = imin_rap_vis / vis_nx
imax_rap_frac = imax_rap_vis / vis_nx

jmin_rap_frac = jmin_rap_vis / vis_ny
jmax_rap_frac = jmax_rap_vis / vis_ny

print(imin_rap_frac,imax_rap_frac)
print(jmin_rap_frac,jmax_rap_frac)

#
##################################################

/Users/wjc/Library/CloudStorage/OneDrive-SDSMT/GOES_Imagery/
0.2206 0.4204
0.0 0.3333333333333333


## Pass One.  Pull all fields and save the ranges.

In [5]:
vmin_arr = np.zeros(17)
vmax_arr = np.zeros(17)


inventory = pd.DataFrame(columns = ['Channel', 'Wavelength', 'Label','Δx','p000','p002','p005','p010','p025','p050','p075','p090','p095','p098','p100'])


for ch_band in range(1,17):

    channel        = ch_band
    ch_band_string = str(ch_band).zfill(2)


    image_date  = datetime.utcnow().date()
    region      =                   'CONUS'



    if (region == 'CONUS') :
        region_lab               = ' SODAK Band '
        png_processing_directory = png_file_dir3
        gif_file_name            = "./SD_BAND_" + ch_band_string + "_"





    # We want to match something like:
    # https://thredds-test.unidata.ucar.edu/thredds/catalog/satellite/goes16/GOES16/Mesoscale-1/Channel08/20181113/catalog.html

    # Construct the data_url string


    data_url = "https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel"+ \
                ch_band_string+"/"+YYYYMMDD+"/OR_ABI-L2-CMIPC-M6C"+ch_band_string+"_G16_"+file_time_string+".nc"

    # Print out your URL and verify it works!

    print(data_url)


    ds  = xr.open_dataset(data_url)


    nx = ds.dims["x"]
    ny = ds.dims["y"]

    imin = round(imin_rap_frac * nx)
    imax = round(imax_rap_frac * nx)

    jmin = round(jmin_rap_frac * ny)
    jmax = round(jmax_rap_frac * ny)


    dat = ds.metpy.parse_cf('Sectorized_CMI')[jmin : jmax,
                                              imin : imax]
    

    directory_name   = datetime.strptime(ds.start_date_time, '%Y%j%H%M%S').strftime("%Y-%m-%d_%H%M")

    if (ch_band >= 7):
        sigfigs = 0
        factor  = 1
    else:
        sigfigs = 2
        factor  = 100
        
        
    rowdf = pd.DataFrame(data = {'Channel'   : ch_band,
                                 'Wavelength': channel_wavelengths[ch_band],
                                 'Label'     : channel_names[ch_band],
                                 'Δx'        : int(round(dat.coords["x"][1].values-dat.coords["x"][0].values,0)),
                                 'p000'      : round(np.nanmin(a=dat.values),sigfigs),
                                 'p002'      : round(np.nanpercentile(a=dat.values, q=2),sigfigs),
                                 'p005'      : round(np.nanpercentile(a=dat.values, q=5),sigfigs),
                                 'p010'      : round(np.nanpercentile(a=dat.values, q=10),sigfigs),
                                 'p025'      : round(np.nanpercentile(a=dat.values, q=25),sigfigs),
                                 'p050'      : round(np.nanpercentile(a=dat.values, q=50),sigfigs),
                                 'p075'      : round(np.nanpercentile(a=dat.values, q=75),sigfigs),
                                 'p090'      : round(np.nanpercentile(a=dat.values, q=90),sigfigs),
                                 'p095'      : round(np.nanpercentile(a=dat.values, q=95),sigfigs),
                                 'p098'      : round(np.nanpercentile(a=dat.values, q=98),sigfigs),
                                 'p100'      : round(np.nanmax(a=dat.values),sigfigs)},
                        index = [ch_band])
    
    inventory = pd.concat([inventory,rowdf])

print("Dataframe for " + directory_name)
display(inventory)
os.system("mkdir -v "+directory_name)


https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel01/20230522/OR_ABI-L2-CMIPC-M6C01_G16_s20231421541170_e20231421541170_c20231421541170.nc
https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel02/20230522/OR_ABI-L2-CMIPC-M6C02_G16_s20231421541170_e20231421541170_c20231421541170.nc
https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel03/20230522/OR_ABI-L2-CMIPC-M6C03_G16_s20231421541170_e20231421541170_c20231421541170.nc
https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel04/20230522/OR_ABI-L2-CMIPC-M6C04_G16_s20231421541170_e20231421541170_c20231421541170.nc
https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel05/20230522/OR_ABI-L2-CMIPC-M6C05_G16_s20231421541170_e20231421541170_c20231421541170.nc
https://thredds.ucar.edu/thred

,Channel,Wavelength,Label,Δx,p000,p002,p005,p010,p025,p050,p075,p090,p095,p098,p100
1,1,0.47,Blue-Visible,1002,0.10,0.14,0.15,0.16,0.18,0.21,0.25,0.44,0.51,0.58,1.00
2,2,0.64,Red-Visible,501,0.04,0.08,0.09,0.10,0.12,0.14,0.18,0.37,0.43,0.51,1.00
3,3,0.86,Vegetation Near-IR,1002,0.03,0.17,0.19,0.20,0.22,0.25,0.31,0.44,0.51,0.59,1.08
4,4,1.37,Cirrus Near-IR,2004,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.04,0.08,0.12,0.39
5,5,1.60,Snow/Ice Near-IR,1002,0.00,0.11,0.14,0.17,0.21,0.24,0.28,0.33,0.37,0.41,0.64
6,6,2.20,Cloud Particle Near-IR,2004,0.01,0.07,0.09,0.11,0.13,0.17,0.21,0.26,0.30,0.33,0.44
7,7,3.90,Middle Infrared,2004,243.00,257.00,270.00,285.00,295.00,299.00,301.00,304.00,306.00,308.00,311.00
8,8,6.20,Upper-Level Water Vapor,2004,218.00,226.00,228.00,229.00,230.00,232.00,235.00,238.00,239.00,240.00,242.00
9,9,6.90,Mid-Level Water Vapor,2004,219.00,230.00,233.00,236.00,239.00,242.00,246.00,249.00,250.00,251.00,253.00
10,10,7.30,Low-Level Water Vapor,2004,219.00,232.00,237.00,242.00,248.00,251.00,256.00,258.00,259.00,260.00,261.00


mkdir: 2023-05-22_1541: File exists


256

## Make Images

In [6]:
##################################################
#
# Control Setup
#

# %load solutions/data_url.py


# Cell content replaced by load magic replacement.

# Create variables for URL generation

for ch_band in range(1,17):

    channel        = ch_band
    ch_band_string = str(ch_band).zfill(2)


    image_date  = datetime.utcnow().date()
    region      =                   'CONUS'



    if (region == 'CONUS') :
        region_lab               = ' SODAK Band '
        png_processing_directory = png_file_dir3
        gif_file_name            = "./SD_BAND_" + ch_band_string + "_"





    # We want to match something like:
    # https://thredds-test.unidata.ucar.edu/thredds/catalog/satellite/goes16/GOES16/Mesoscale-1/Channel08/20181113/catalog.html

    # Construct the data_url string


    data_url = "https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel"+ \
                ch_band_string+"/"+YYYYMMDD+"/OR_ABI-L2-CMIPC-M6C"+ch_band_string+"_G16_"+file_time_string+".nc"

    # Print out your URL and verify it works!

    print(data_url)


    ds  = xr.open_dataset(data_url)


    nx = ds.dims["x"]
    ny = ds.dims["y"]

    imin = round(imin_rap_frac * nx)
    imax = round(imax_rap_frac * nx)

    jmin = round(jmin_rap_frac * ny)
    jmax = round(jmax_rap_frac * ny)


    dat = ds.metpy.parse_cf('Sectorized_CMI')[jmin : jmax,
                                              imin : imax]



    x    = dat['x']       
    y    = dat['y']
    proj = dat.metpy.cartopy_crs

    tz         = 'America/Denver'
    time_utc   = datetime.strptime(ds.start_date_time, '%Y%j%H%M%S')
    valid_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").strftime("%Y-%m-%d %H:%M:%S %Z")
    local_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").tz_convert(tz=tz).strftime("%Y-%m-%d %H:%M:%S %Z")

    file_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").strftime("%Y-%m-%d_%H%M")

    print(valid_time,local_time, file_time)

    image_header_label = "GOES 16" + region_lab + str(channel)+ channel_lab[channel]



    ny = dat.shape[0]
    nx = dat.shape[1]      
    alpha2d = np.sqrt(np.outer(np.abs(np.hanning(ny)),np.abs(np.hanning(nx))))
    alpha2d = np.where(alpha2d>alpha_factor,alpha_factor,alpha2d)
    alpha2d = alpha2d / alpha_factor




    fig = plt.figure(figsize=(8, 8), facecolor = 'white')

    plt.suptitle(image_header_label,
                 fontsize = 20, 
                 color    = Mines_Blue)
    ax = fig.add_subplot(1, 1, 1, projection=proj)
    ax.set_title(valid_time + "  (" + local_time+")",
                 fontsize =      15, 
                 color    = Mines_Blue)
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=1, edgecolor=Mines_Blue)
    ax.add_feature(cfeature.STATES.with_scale('50m'),    linestyle=':', edgecolor=Mines_Blue)
    ax.add_feature(cfeature.BORDERS.with_scale('50m'),   linewidth=1, edgecolor=Mines_Blue)


    
    if (ch_band >= 7):
        colortab  = "gist_ncar"
        bar_label = "Top-of-Atmosphere Radiometric Temperature (K)"
        vmin = inventory.loc[7:]['p002'].values.min()
        vmax = inventory.loc[7:]['p098'].values.max()

    else:
        colortab  = "Greys_r"
        bar_label = "Top-of-Atmosphere Reflectance (fraction)"
        vmin = 0.
        vmax = 0.3
        
    im = ax.imshow(                       dat, 
                   extent = (x.min(), x.max(), 
                             y.min(), y.max()), 
                   origin =            'upper',
                   cmap   =          colortab,alpha = alpha2d,
                   vmin   =              vmin,
                   vmax   =              vmax)


    plt.colorbar(im,
                ax=ax,
                orientation = "horizontal",
                pad = 0,
                label = bar_label)
    
    


    #########################################
    #
    # Insert a Clock
    #

    axins = fig.add_axes(rect     =    [0.002,
                                        0.825,
                                        0.12,
                                        0.12],
                          projection  =  "polar")

    time_for_clock = pd.to_datetime(time_utc).tz_localize(tz="UTC").tz_convert(tz=tz).time()

    hour   = time_for_clock.hour
    minute = time_for_clock.minute
    second = time_for_clock.second

    circle_theta  = np.deg2rad(np.arange(0,360,0.01))
    circle_radius = circle_theta * 0 + 1

    if (hour > 12) :
        hour = hour - 12

    angles_h = 2*np.pi*hour/12+2*np.pi*minute/(12*60)+2*second/(12*60*60)
    angles_m = 2*np.pi*minute/60+2*np.pi*second/(60*60)

    #print(time_for_clock)
    #print(hour,   np.rad2deg(angles_h))
    #print(minute, np.rad2deg(angles_m))


    plt.setp(axins.get_yticklabels(), visible=False)
    plt.setp(axins.get_xticklabels(), visible=False)
    axins.spines['polar'].set_visible(False)
    axins.set_ylim(0,1)
    axins.set_theta_zero_location('N')
    axins.set_theta_direction(-1)
    axins.set_facecolor("white")
    axins.grid(False)

    axins.plot([angles_h,angles_h], [0,0.60], color=Mines_Blue, linewidth=1.5)
    axins.plot([angles_m,angles_m], [0,0.95], color=Mines_Blue, linewidth=1.5)
    axins.plot(circle_theta, circle_radius, color=Mines_Blue, linewidth=1)

    #
    #########################################



    #########################################
    #
    # Insert a Footprint Map
    #   


    axmap = fig.add_axes(rect        =    [0.88, 
                                           0.825,
                                           0.12, 
                                           0.12],
                         projection = proj)

    axmap.add_feature(cfeature.COASTLINE, linewidth=0.5, edgecolor=Mines_Blue)

    footprint_xy=np.array([[x.min(),y.min()],
                           [x.min(),y.max()],
                           [x.max(),y.max()],
                           [x.max(),y.min()],
                           [x.min(),y.min()]])

    footprint = patches.Polygon(xy        = footprint_xy,
                                facecolor =        'c')


    axmap.add_patch(footprint)

    axmap.set_global()

    #
    #########################################

    ax.set_frame_on(False)
    plt.tight_layout()
    dataset_png_file_name = directory_name +  "/SODAK_CH" + ch_band_string + "_" + file_time
    plt.savefig( dataset_png_file_name,
                    facecolor   = 'white', 
                    transparent =   False)
    plt.close()



#
##################################################

https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel01/20230522/OR_ABI-L2-CMIPC-M6C01_G16_s20231421541170_e20231421541170_c20231421541170.nc
2023-05-22 15:41:17 UTC 2023-05-22 09:41:17 MDT 2023-05-22_1541


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_18003/2640808159.py:217: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel02/20230522/OR_ABI-L2-CMIPC-M6C02_G16_s20231421541170_e20231421541170_c20231421541170.nc
2023-05-22 15:41:17 UTC 2023-05-22 09:41:17 MDT 2023-05-22_1541


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_18003/2640808159.py:217: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel03/20230522/OR_ABI-L2-CMIPC-M6C03_G16_s20231421541170_e20231421541170_c20231421541170.nc
2023-05-22 15:41:17 UTC 2023-05-22 09:41:17 MDT 2023-05-22_1541


...

/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_18003/2640808159.py:217: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel15/20230522/OR_ABI-L2-CMIPC-M6C15_G16_s20231421541170_e20231421541170_c20231421541170.nc
2023-05-22 15:41:17 UTC 2023-05-22 09:41:17 MDT 2023-05-22_1541


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_18003/2640808159.py:217: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)


https://thredds.ucar.edu/thredds/dodsC/satellite/goes/east/products/CloudAndMoistureImagery/CONUS/Channel16/20230522/OR_ABI-L2-CMIPC-M6C16_G16_s20231421541170_e20231421541170_c20231421541170.nc
2023-05-22 15:41:17 UTC 2023-05-22 09:41:17 MDT 2023-05-22_1541


/var/folders/0c/8xbkg06s6cz0yy2505h06nlm00017d/T/ipykernel_18003/2640808159.py:217: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/Users/wjc/miniconda3/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
